## ⬇️ Instalaciones y carga del tokenizer



In [ ]:
%%capture
!pip install transformers
!pip install datasets
!pip install bitsandbytes
!pip install trl
!pip install wanb

In [ ]:
# Log in to W&B account
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: francoartico0 (francoartico0-universidad-nacional-de-c-rdoba) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [ ]:
from datasets import load_dataset, Dataset
from google.colab import userdata
from transformers import AutoTokenizer

access_token = userdata.get('HUGGINGFACE_ACCESS_TOKEN')

checkpoint = "meta-llama/Llama-3.1-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(checkpoint,
                                          token=access_token,
                                          return_tensors="pt"
)

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

## 📁 Cargando conjunto de datos

In [ ]:
from google.colab import userdata, drive
drive.mount('/content/drive')
dataset = load_dataset("/content/drive/MyDrive/SQAC", split="train", trust_remote_code=True) # Dataset's hub has an issue

Mounted at /content/drive


train.json:   0%|          | 0.00/11.0M [00:00<?, ?B/s]

dev.json:   0%|          | 0.00/1.40M [00:00<?, ?B/s]

test.json:   0%|          | 0.00/1.35M [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

## 🔍 Inspeccionando el conjunto de datos

In [ ]:
print(dataset)

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 15036
})


In [ ]:
dataset.features

{'id': Value(dtype='string', id=None),
 'title': Value(dtype='string', id=None),
 'context': Value(dtype='string', id=None),
 'question': Value(dtype='string', id=None),
 'answers': Sequence(feature={'text': Value(dtype='string', id=None), 'answer_start': Value(dtype='int32', id=None)}, length=-1, id=None)}

In [ ]:
contexts = dataset[:5]
questions = dataset['question'][:5]

In [ ]:
contexts

{'id': ['6cf3dcd6-b5a3-4516-8f9e-c5c1c6b66628',
  '2663226e-e652-43a2-a6ba-c1fd02a1df31',
  '02a1fd0a-b730-4b95-8b07-664d5ac9bd91',
  'a60598d9-70e0-4ccc-a901-a5d21b6aca26',
  'da6bfa6b-d310-4613-b262-465eb95e1724'],
 'title': ['Historia de Japón',
  'Historia de Japón',
  'Historia de Japón',
  'CESS-CAST-A_18964_20000923_rec.txt',
  'CESS-CAST-A_18964_20000923_rec.txt'],
 'context': ['La historia de Japón (日本の歴史 o 日本史, Nihon no rekishi / Nihonshi?) es la sucesión de hechos acontecidos dentro del archipiélago japonés. Algunos de estos hechos aparecen aislados e influenciados por la naturaleza geográfica de Japón como nación insular, en tanto que otra serie de hechos, obedece a influencias foráneas como en el caso del Imperio chino, el cual definió su idioma, su escritura y, también, su cultura política. Asimismo, otra de las influencias foráneas fue la de origen occidental, lo que convirtió al país en una nación industrial, ejerciendo con ello una esfera de influencia y una expansión 

In [ ]:
questions

['¿Qué influencia convirtió Japón en una nación industrial?',
 '¿Cuándo se detuvo el expansionismo de Japón?',
 '¿Quién definió el idioma, la escritura y la cultura política del Japón?',
 '¿Qué premio ganó Margarita Fullana?',
 '¿Quién obtuvo la medalla de plata?']

## ⏳ Preprocesamiento de datos

In [ ]:
def format_conversation(row):
    user_message = {"role": "user", "content": "Genera exactamente %s preguntas de lectura comprensiva, sin incluir las respuestas ni opciones, sobre el siguiente texto: %s" % (row['num_questions'], row['context'])}
    assistant_message = {"role": "assistant", "content": "Aquí tienes algunas preguntas sobre el texto proporcionado \n %s" % row['questions']}
    row["messages"] = [user_message, assistant_message]
    return row

In [ ]:
# Remove unnecessary columns
dataset = dataset.remove_columns(['id','title', 'answers'])
dataset = dataset.filter(lambda row: row["context"].startswith("Biografía"))

Filter:   0%|          | 0/15036 [00:00<?, ? examples/s]

In [ ]:
df = dataset.to_pandas()

# Grouping contexts that are the same and enumerate questions for make the prompt
grouped_df = (
    df.groupby('context')
    .agg(
        questions = ('question', lambda qs: "\n".join([f"{i+1}. {q}" for i, q in enumerate(qs)])),
        num_questions = ('question', 'count')
    )
    .reset_index()
)

grouped_df

,context,questions,num_questions
0,"Biografía \nAndrade nació en São Paulo, ciudad...",1. ¿Dónde pasó gran parte de su vida Mário de ...,3
1,Biografía \nAriadna Thalía Sodi Miranda nació ...,1. ¿Cómo se llama realmente Thalía?\n2. ¿Cuál ...,3
2,Biografía \nBall fue hija de Henry Durrell Bal...,1. ¿Cuál era el apodo de la madre de Ball?\n2....,3
3,Biografía \nDavid Bowie nació como David Rober...,1. ¿Cuál era el sobrenombre de la madre de Bow...,3
4,"Biografía \nDe forma similar a su creación, la...",1. ¿Cuándo se presentó por primera vez el Joke...,3
5,"Biografía \nDespués de que su padre, el empera...",1. ¿Quién era el padre de Xuanye?\n2. ¿Cuál er...,3
6,Biografía \nEdwin Eugene Aldrin Jr. nació el 2...,1. ¿Qué día nació Edwin Eugene Aldrin Jr.?\n2....,3
7,Biografía \nEl primer antepasado masculino con...,1. ¿Qué relación guardaba Sebastian List con F...,3
8,Biografía \nFranz Peter Schubert nació en Himm...,1. ¿Cuál era la religión que profesaba Schuber...,3
9,Biografía \nFrédéric Chopin nació en la aldea ...,1. ¿Cuándo afirmaba Chopin haber nacido?\n2. ¿...,3


In [ ]:
# Convert df to dataset
cq_dataset = Dataset.from_pandas(grouped_df)

# Convert dataset of cq in dataset of conversations
conversation_dataset = cq_dataset.map(format_conversation)
conversation_dataset = conversation_dataset.remove_columns(['context', 'questions', 'num_questions'])
conversation_dataset['messages'][:5]

Map:   0%|          | 0/52 [00:00<?, ? examples/s]

[[{'content': 'Genera exactamente 3 preguntas de lectura comprensiva, sin incluir las respuestas ni opciones, sobre el siguiente texto: Biografía \nAndrade nació en São Paulo, ciudad en la que vivió durante prácticamente toda su vida, en el número 320 de la Rua Aurora, donde residían sus padres, Carlos Augusto de Moraes Andrade y Maria Luísa Leite Moraes Andrade. En su infancia fue considerado un niño prodigio como pianista. Al mismo tiempo, estudiaba historia, arte y, especialmente, poesía. Tenía un sólido dominio de la lengua francesa, y leyó durante su infancia a Rimbaud y a los principales poetas simbolistas franceses. Aunque escribió poesía desde su más tierna infancia (su primer poema data de 1904), su primera vocación fue la música, y en 1911 se matriculó en el Conservatorio de São Paulo.',
   'role': 'user'},
  {'content': 'Aquí tienes algunas preguntas sobre el texto proporcionado \n 1. ¿Dónde pasó gran parte de su vida Mário de Andrade?\n2. ¿En qué calle vivían los padres de 

## 🏋️ Carga y entrenamiento del modelo

In [ ]:
from transformers import TrainingArguments, AutoModelForCausalLM, BitsAndBytesConfig
import torch

# Get the nf4 model
nf4_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_quant_type="nf4",
   bnb_4bit_use_double_quant=True,
   bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(checkpoint,
                                             device_map="auto",
                                             token=access_token,
                                             quantization_config=nf4_config
)

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

In [ ]:
from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [ ]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "up_proj", "down_proj", "o_proj", "gate_proj"]

)

model = get_peft_model(model, lora_config)

In [ ]:
# Get number of parameters
model.print_trainable_parameters()

trainable params: 20,971,520 || all params: 8,051,232,768 || trainable%: 0.2605


In [ ]:
from transformers import TrainingArguments
from trl import SFTConfig, SFTTrainer

tokenizer.pad_token = tokenizer.eos_token

sft_config = SFTConfig(
    learning_rate=3e-4,
    max_seq_length=2048,
    max_steps=120,
    packing=True,
    optim="adamw_8bit",
    output_dir="output",
    weight_decay=0.01,
    warmup_steps=10,
    logging_steps=5,
    per_device_train_batch_size=1,
    seed=42,
    push_to_hub=True,
    save_strategy="epoch",
    hub_model_id="spanish-question-generator",
    hub_token=access_token,
    fp16=True,
    run_name="plain messages"
)

trainer = SFTTrainer(
    model=model,
    train_dataset=conversation_dataset,
    peft_config=lora_config,
    args=sft_config,
)

trainer.train()

Converting train dataset to ChatML:   0%|          | 0/52 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/52 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/52 [00:00<?, ? examples/s]

Packing train dataset:   0%|          | 0/52 [00:00<?, ? examples/s]

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
5,1.818900
10,1.628900
15,1.425500
20,1.219200
25,1.031400
30,0.976400
35,0.649100
40,0.671200
45,0.400600
50,0.349500


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/

TrainOutput(global_step=120, training_loss=0.4589766050999363, metrics={'train_runtime': 1060.5142, 'train_samples_per_second': 0.113, 'train_steps_per_second': 0.113, 'total_flos': 1.0771183044329472e+16, 'train_loss': 0.4589766050999363})

In [ ]:
save_path = "/content/drive/MyDrive/adapters"
model.save_pretrained(save_path)

In [ ]:
trainer.push_to_hub("End of training")

events.out.tfevents.1740511589.1f8dc92a3392.453.0:   0%|          | 0.00/13.0k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Doberfran/spanish-question-generator/commit/c8694ab2be99aeddf5cd95b881efcc0555ba48e5', commit_message='End of training', commit_description='', oid='c8694ab2be99aeddf5cd95b881efcc0555ba48e5', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Doberfran/spanish-question-generator', endpoint='https://huggingface.co', repo_type='model', repo_id='Doberfran/spanish-question-generator'), pr_revision=None, pr_num=None)